In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
train = pd.read_csv('train.csv')

# Train & Validation set

In [7]:
train["Employment_Type"].fillna("Other", inplace = True)

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train.Employment_Type = le.fit_transform(train.Employment_Type)

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train.PERFORM_CNS_SCORE_DESCRIPTION = le.fit_transform(train.PERFORM_CNS_SCORE_DESCRIPTION)

In [10]:
#convert year into month
def change_col_month(col):
    year = int(col.split()[0].replace('yrs',''))
    month = int(col.split()[1].replace('mon',''))
    return year*12+month

train['CREDIT_HISTORY_LENGTH'] = train['CREDIT_HISTORY_LENGTH'].apply(change_col_month)
train['AVERAGE_ACCT_AGE'] = train['AVERAGE_ACCT_AGE'].apply(change_col_month)

In [11]:
#
def age(dob):
    yr = int(dob[-2:])
    if yr >=0 and yr < 20:
        return yr + 2000
    else:
         return yr + 1900
        
train['Date_of_Birth'] = train['Date_of_Birth'].apply(age)
train['DisbursalDate'] = train['DisbursalDate'].apply(age)
train['Age']=train['DisbursalDate']-train['Date_of_Birth']
train= train.drop(['DisbursalDate','Date_of_Birth'],axis=1)


In [12]:
train= train.drop(['UniqueID'], axis = 1)
train.head()

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment_Type,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_NO_OF_ACCTS,PRI_ACTIVE_ACCTS,PRI_OVERDUE_ACCTS,PRI_CURRENT_BALANCE,PRI_SANCTIONED_AMOUNT,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_ACTIVE_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,loan_default,Age
0,50578,58400,89.55,67,22807,45,1441,1,6,1998,1,1,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34
1,47145,65550,73.23,67,22807,45,1502,2,6,1998,1,1,0,0,0,0,598,8,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,23,23,0,1,33
2,53278,61360,89.63,67,22807,45,1497,2,6,1998,1,1,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33
3,57513,66113,88.48,67,22807,45,1501,2,6,1998,1,1,0,0,0,0,305,11,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,8,15,1,1,25
4,52378,60300,88.39,67,22807,45,1495,2,6,1998,1,1,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,41


In [13]:
X = train.drop(['loan_default'], axis = 1)
y = train['loan_default']

# Train set and Validation set

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2, random_state = 0)

# train on XG boost

In [16]:
!pip install xgboost

In [17]:
 from xgboost import XGBClassifier

In [18]:
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

# Confusion Matrix for validation Values

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_validation)
cm = confusion_matrix(y_validation, y_pred)
print(cm)
accuracy_score(y_validation, y_pred)

[[36214   400]
 [ 9707   310]]


0.7832557740558855

# Confusion Matrix for train Values

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_train_pred = classifier.predict(X_train)
cm = confusion_matrix(y_train, y_train_pred)
print(cm)
accuracy_score(y_train, y_train_pred)

[[145167    762]
 [ 37808   2786]]


0.7932158500560252

# test set

In [23]:
test = pd.read_csv('test_bqCt9Pv.csv')

In [24]:
#Checking Missing Values
print(test.isnull().sum())


UniqueID                                  0
disbursed_amount                          0
asset_cost                                0
ltv                                       0
branch_id                                 0
supplier_id                               0
manufacturer_id                           0
Current_pincode_ID                        0
Date_of_Birth                             0
Employment_Type                        3443
DisbursalDate                             0
State_ID                                  0
Employee_code_ID                          0
MobileNo_Avl_Flag                         0
Aadhar_flag                               0
PAN_flag                                  0
VoterID_flag                              0
Driving_flag                              0
Passport_flag                             0
PERFORM_CNS_SCORE                         0
PERFORM_CNS_SCORE_DESCRIPTION             0
PRI_NO_OF_ACCTS                           0
PRI_ACTIVE_ACCTS                

In [25]:
# Checking unique values
print("Distinct Emp Type :",test["Employment_Type"].unique())
#Checking missing valus in percentage
print("Missing Emp Type %:",test["Employment_Type"].isnull().sum() / len(test) * 100)

Distinct Emp Type : ['Salaried' 'Self employed' nan]
Missing Emp Type %: 3.063385294326998


In [26]:
# Imputing Employment.Type as Other
test["Employment_Type"].fillna("Other", inplace = True)

In [27]:
#cross checking
print("Distinct Emp Type :",test["Employment_Type"].unique())
print("Missing Emp Type %:",test["Employment_Type"].isnull().sum() / len(test) * 100)

Distinct Emp Type : ['Salaried' 'Self employed' 'Other']
Missing Emp Type %: 0.0


In [28]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
test.Employment_Type = le.fit_transform(test.Employment_Type)

In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
test.PERFORM_CNS_SCORE_DESCRIPTION = le.fit_transform(test.PERFORM_CNS_SCORE_DESCRIPTION)

In [30]:
#convert year into month
def change_col_month(col):
    year = int(col.split()[0].replace('yrs',''))
    month = int(col.split()[1].replace('mon',''))
    return year*12+month

test['CREDIT_HISTORY_LENGTH'] = test['CREDIT_HISTORY_LENGTH'].apply(change_col_month)
test['AVERAGE_ACCT_AGE'] = test['AVERAGE_ACCT_AGE'].apply(change_col_month)

In [31]:
#calculate age
def age(dob):
    yr = int(dob[-2:])
    if yr >=0 and yr < 20:
        return yr + 2000
    else:
         return yr + 1900
        
test['Date_of_Birth'] = test['Date_of_Birth'].apply(age)
test['DisbursalDate'] = test['DisbursalDate'].apply(age)
test['Age']=test['DisbursalDate']-test['Date_of_Birth']
test= test.drop(['DisbursalDate','Date_of_Birth'],axis=1)

test.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment_Type,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_NO_OF_ACCTS,PRI_ACTIVE_ACCTS,PRI_OVERDUE_ACCTS,PRI_CURRENT_BALANCE,PRI_SANCTIONED_AMOUNT,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_ACTIVE_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,Age
0,655269,53478,63558,86.54,67,22807,45,1497,1,6,1998,1,1,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,44
1,723482,55513,63163,89.45,67,22807,45,1497,2,6,1998,1,1,0,0,0,0,749,2,2,1,0,43898,48780,48780,0,0,0,0,0,0,5605,0,1,0,8,12,1,33
2,758529,65282,84320,79.93,78,23135,86,2071,1,4,1646,1,1,1,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23
3,763449,46905,63896,76.58,78,17014,45,2070,2,4,1646,1,1,0,0,0,0,14,17,1,1,1,132480,255000,255000,0,0,0,0,0,0,0,0,0,0,29,29,0,45
4,708663,51428,63896,86.08,78,17014,45,2069,1,4,1646,1,1,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46


In [32]:
test = test.drop(['UniqueID'], axis = 1)
test.head()

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment_Type,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_NO_OF_ACCTS,PRI_ACTIVE_ACCTS,PRI_OVERDUE_ACCTS,PRI_CURRENT_BALANCE,PRI_SANCTIONED_AMOUNT,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_ACTIVE_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,Age
0,53478,63558,86.54,67,22807,45,1497,1,6,1998,1,1,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,44
1,55513,63163,89.45,67,22807,45,1497,2,6,1998,1,1,0,0,0,0,749,2,2,1,0,43898,48780,48780,0,0,0,0,0,0,5605,0,1,0,8,12,1,33
2,65282,84320,79.93,78,23135,86,2071,1,4,1646,1,1,1,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23
3,46905,63896,76.58,78,17014,45,2070,2,4,1646,1,1,0,0,0,0,14,17,1,1,1,132480,255000,255000,0,0,0,0,0,0,0,0,0,0,29,29,0,45
4,51428,63896,86.08,78,17014,45,2069,1,4,1646,1,1,0,0,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46


# Value Predicted 

In [35]:
test_pred =classifier.predict(test)
test_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)